## Projet BigData/BI

Réalisé par : ABDENNROUI Ilyes / LAURENT Emmanuel / LEGRAND Antoine / RONGIER Thomas

In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline, make_union
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import xgboost as xgb
pd.set_option('display.max_columns', None)

## 1. Introduction 

Dans un contexte ou les talents travaillant dans le domaine de l'immobilier, sont de plus en plus rare, nous avons été mandaté par l'agence immobilière SIAD Immo pour trouver une solution à ce problème de recrutement à l'aide de technique de Machine Learning. Nous avons donc, en accord avec l'entreprise, décidé de mettre en place un outil permettant aux clients de faire estimer leurs biens de façon automatique moyennant une contrepartie financière. Cet outil permettra à l'entreprise de se passer d'une main d'oeuvre qui est de plus en plus difficile à recruter et permettra aux clients de gagner plus de temps. En effet, les estimations se feront de manière instantanée, il n'y aura plus de contrainte de rendez-vous ou de durée d'estimation.  

Pour nous permettre de réaliser notre projet, l'entreprise a mis à notre disposition un jeu de données contenant différentes informations sur 2919 logmements. Ces informations nous renseingent sur la surface du bien, l'état général, la date de construction... En tout et pour tout nous avons à disposition 81 variables.

Notre but, est de mettre en place un modèle de machine learning qui permet d'estimer le prix d'un bien en fonction des caractéristiques de ce dernier.

### Dictionnaire des données

In [68]:
dico = pd.read_excel('C:/Users/100518/Desktop/Projet_BI/equipe1-projetbigdatabi2022/dictionnaire_de_données.xlsx')

In [69]:
dico

,Nom champ,Type de données,Longueur du champ,Contrainte,Description
0,Id,Integer,NaN,Primary Key,Identifiant de la ligne
1,MSSubClass,Integer,NaN,Not Null,La classe du bâtiment
2,MSZoning,Varchar,20.0,Not Null,La classe de la zone du bâtiment
3,LotFrontage,Integer,NaN,NaN,Distance en pied reliant la propriété à la rue...
4,LotArea,Integer,NaN,Not Null,Taille de la propriété en pieds carrés
...,...,...,...,...,...
75,MoSold,Integer,NaN,Not Null,Mois vendu (MM)
76,YrSold,Integer,NaN,Not Null,Année de vente (AAAA)
77,SaleType,Varchar,20.0,Not Null,Type de vente
78,SaleCondition,Varchar,20.0,Not Null,Condition de vente


### Importation des données

In [70]:
df_train = pd.read_csv("C:/Users/100518/Desktop/Projet_BI/equipe1-projetbigdatabi2022/Data/train.csv")
df_test = pd.read_csv("C:/Users/100518/Desktop/Projet_BI/equipe1-projetbigdatabi2022/Data/test.csv")
df_test_sub = pd.read_csv("C:/Users/100518/Desktop/Projet_BI/equipe1-projetbigdatabi2022/Data/sample_submission.csv")

#Dans le dataset de test, il manque la variable cible. Cette dernière, se trouve dans un autre dataset
#Il est donc nécessaire de faire une jointure entre ces 2 datasets
#Pour récupérer les informations
df_test = pd.merge(df_test, df_test_sub, how='inner', left_on = 'Id', right_on = 'Id')

In [71]:
#Sur Kaggle, le dataset a déjà été découpé en train et test set. 
#Néanmoins nous ne savons pas de quelle manière ce découpage a été fait.
#Nous allons donc concatener les 2 dataset pour réaliser le découpage nous même..
df = df_train

In [72]:
#Apperçu de notre jeu de données après la concatenation 
df.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,Gas

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [74]:
df.shape

(1460, 81)

## 2. Description des données 

1460 observations et 81 variables caractérisant les biens immobiliers.

### Selection des variables 

Hypothèses : 

In [122]:
df2 = df[["Id","MSSubClass","MSZoning","LotFrontage","LotArea","Neighborhood","OverallQual","OverallCond",
"YearBuilt","YearRemodAdd","MasVnrArea","ExterQual","BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF",
"LowQualFinSF","GrLivArea","BsmtFullBath","FullBath","HalfBath","BedroomAbvGr","KitchenAbvGr","TotRmsAbvGrd",
"Fireplaces","GarageYrBlt","GarageCars","GarageArea","WoodDeckSF",
"OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea","MiscVal","MoSold","YrSold","SalePrice"]]

In [123]:
df2

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Neighborhood,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,RL,65.0,8450,CollgCr,7,5,2003,2003,196.0,Gd,150,856,856,854,0,1710,1,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500
1,2,20,RL,80.0,9600,Veenker,6,8,1976,1976,0.0,TA,284,1262,1262,0,0,1262,0,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500
2,3,60,RL,68.0,11250,CollgCr,7,5,2001,2002,162.0,Gd,434,920,920,866,0,1786,1,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500
3,4,70,RL,60.0,9550,Crawfor,7,5,1915,1970,0.0,TA,540,756,961,756,0,1717,1,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000
4,5,60,RL,84.0,14260,NoRidge,8,5,2000,2000,350.0,Gd,490,1145,1145,1053,0,2198,1,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Gilbert,6,5,1999,2000,0.0,TA,953,953,953,694,0,1647,0,2,1,3,1,7,1,1999.0,2,460,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,RL,85.0,13175,NWAmes,6,6,1978,1988,119.0,TA,589,1542,2073,0,0,2073,1,2,0,3,1,7,2,1978.0,2,500,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,RL,66.0,9042,Crawfor,7,9,1941,2006,0.0,Ex,877,1152,1188,1152,0,2340,0,2,0,4,1,9,2,1941.0,1,252,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,RL,68.0,9717,NAmes,5,6,1950,1996,0.0,TA,0,1078,1078,0,0,1078,1,1,0,2,1,5,0,1950.0,1,240,366,0,112,0,0,0,0,4,2010,142125


### Renommer les variables : A FAIRE

In [124]:
# On renomme les modalités ayant un nom peu explicite
df2.rename(columns={'Id': 'ID',
                   "MSSubClass" : "Type_propri",
                    "MSZoning": "Classe_zone_propri",
                    "LotFrontage": "Dist_metre_rue",
                    "LotArea" : "Taille_propri_m2",
                    "Neighborhood" : "Quartier",
                    "OverallQual": "Eval_mater_finition",
                    "OverallCond": "Eval_etat_gene",
                    "YearBuilt": "Annee_construct",
                    "YearRemodAdd": "Annee_renov",
                    "MasVnrArea" : "Type_placage_m2",
                    "ExterQual" : "Eval_mater_ext",
                    "TotalBsmtSF": 'm2_sous-sol',
                    "BsmtUnfSF" : "Non_fini_m2_SS",
                    "1stFlrSF" : "Surf_m2_etage1",
                    "2ndFlrSF" : "Surf_m2_etage2",
                    "LowQualFinSF" : "basse_quali_m2",
                    "GrLivArea" : "Surf_habit_m2",
                    "BsmtFullBath" : "Nb_salledebain_SS",
                    "BsmtHalfBath" : "Nb_demi_salledebain_SS",
                    "FullBath" : "Nb_salledebain",
                    "HalfBath" : "Nb_demi_salledebain",
                    "BedroomAbvGr" : "Nb_chambre",
                    "KitchenAbvGr" : "Nb_cuisine",
                    "TotRmsAbvGrd" : "Nb_piece",
                    "Fireplaces" : "Nb_foyer",
                    "GarageYrBlt" : "Annee_construct_garage",
                    "GarageCars" : "Nb_voiture_garage",
                    "GarageArea" : "Surf_garage_m2",
                    "WoodDeckSF" : "Surf_terrasse_m2",
                    "OpenPorchSF" : "Surf_porche_ouvert_m2",
                    "EnclosedPorch" : "Surf_porche_ferme_m2",
                    "3SsnPorch" : "Surf_porche_3S_m2",
                    "ScreenPorch" : "Surf_veranda_m2",
                    "PoolArea" : "Surf_piscine_m2",
                    "MiscVal" : "Valeur_caract_ext",
                    "MoSold" : "Mois_vente",
                    "YrSold" : "Annee_vente",
                    "SalePrice" : "Prix_vente",
                   },
                   inplace=True)

C:\Users\100518\AppData\Local\Temp\ipykernel_2052\3312216099.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns={'Id': 'ID',


In [125]:
#La fonction describe nous permet de décrire de manière très rapide les différents informations 
#sur les variables quantitatives (moyenne, écart-type, val min et max...)
#Nous permettant ainsi de détecter d'enventuels problèmes.
df2.describe()

,ID,Type_propri,Dist_metre_rue,Taille_propri_m2,Eval_mater_finition,Eval_etat_gene,Annee_construct,Annee_renov,Type_placage_m2,Non_fini_m2_SS,m2_sous-sol,Surf_m2_etage1,Surf_m2_etage2,basse_quali_m2,Surf_habit_m2,Nb_salledebain_SS,Nb_salledebain,Nb_demi_salledebain,Nb_chambre,Nb_cuisine,Nb_piece,Nb_foyer,Annee_construct_garage,Nb_voiture_garage,Surf_garage_m2,Surf_terrasse_m2,Surf_porche_ouvert_m2,Surf_porche_ferme_m2,Surf_porche_3S_m2,Surf_veranda_m2,Surf_piscine_m2,Valeur_caract_ext,Mois_vente,Annee_vente,Prix_vente
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## 3. Préparation des données

### Répartition des variables qualitativres

In [126]:
#Fréquence d'apparition des différentes modalités des variables qualitatives
#C'est à dire les variables de type 'object'

#Copie du dataset de base en ne gardant que les variables du type 'object'
df_object = df2.select_dtypes(include=['object']).copy()

#Fréquence d'apparition de chaque modalité pour les variables qualitatives
for col in list(df_object.columns):
    print('la variable ' + col +' a '+ str(df_object[col].nunique()) +' modalités :\n ')
    print(df2[col].value_counts(normalize=False).map(str) + ' soit: '+round(df2[col].value_counts(normalize=True)*100,2).map(str)+'%')
    print('\n')

la variable Classe_zone_propri a 5 modalités :
 
RL         1151 soit: 78.84%
RM          218 soit: 14.93%
FV            65 soit: 4.45%
RH             16 soit: 1.1%
C (all)       10 soit: 0.68%
Name: Classe_zone_propri, dtype: object


la variable Quartier a 25 modalités :
 
NAmes      225 soit: 15.41%
CollgCr    150 soit: 10.27%
OldTown     113 soit: 7.74%
Edwards     100 soit: 6.85%
Somerst      86 soit: 5.89%
Gilbert      79 soit: 5.41%
NridgHt      77 soit: 5.27%
Sawyer       74 soit: 5.07%
NWAmes        73 soit: 5.0%
SawyerW      59 soit: 4.04%
BrkSide      58 soit: 3.97%
Crawfor      51 soit: 3.49%
Mitchel      49 soit: 3.36%
NoRidge      41 soit: 2.81%
Timber        38 soit: 2.6%
IDOTRR       37 soit: 2.53%
ClearCr      28 soit: 1.92%
StoneBr      25 soit: 1.71%
SWISU        25 soit: 1.71%
MeadowV      17 soit: 1.16%
Blmngtn      17 soit: 1.16%
BrDale        16 soit: 1.1%
Veenker      11 soit: 0.75%
NPkVill       9 soit: 0.62%
Blueste       2 soit: 0.14%
Name: Quartier, dtype: o

### Les valeurs manquantes 

In [127]:
# Compter le nombre d'observations manquantes pour chaque colonne
missing_values = df2.isna().sum()

# Afficher le nombre d'observations manquantes pour chaque colonne
print(missing_values)

missing_values_percent = round((df2.isna().sum() / len(df2)) * 100,2)
print(missing_values_percent.to_string())

ID                          0
Type_propri                 0
Classe_zone_propri          0
Dist_metre_rue            259
Taille_propri_m2            0
Quartier                    0
Eval_mater_finition         0
Eval_etat_gene              0
Annee_construct             0
Annee_renov                 0
Type_placage_m2             8
Eval_mater_ext              0
Non_fini_m2_SS              0
m2_sous-sol                 0
Surf_m2_etage1              0
Surf_m2_etage2              0
basse_quali_m2              0
Surf_habit_m2               0
Nb_salledebain_SS           0
Nb_salledebain              0
Nb_demi_salledebain         0
Nb_chambre                  0
Nb_cuisine                  0
Nb_piece                    0
Nb_foyer                    0
Annee_construct_garage     81
Nb_voiture_garage           0
Surf_garage_m2              0
Surf_terrasse_m2            0
Surf_porche_ouvert_m2       0
Surf_porche_ferme_m2        0
Surf_porche_3S_m2           0
Surf_veranda_m2             0
Surf_pisci

In [128]:
#On supprime la variable LotFrontage car il y a plus de 15% de valeurs manquantes
df2.drop(columns=["Dist_metre_rue"], inplace=True)

C:\Users\100518\AppData\Local\Temp\ipykernel_2052\2757152853.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=["Dist_metre_rue"], inplace=True)


In [129]:
#On supprimer les observations ayant des valeurs manquantes pour les variables suivantes : 
df2.dropna(subset=['Annee_construct_garage','Type_placage_m2'], inplace=True)

C:\Users\100518\AppData\Local\Temp\ipykernel_2052\896359733.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(subset=['Annee_construct_garage','Type_placage_m2'], inplace=True)


In [130]:
df2.shape

(1371, 37)

### Les variables discrètes

In [131]:
df_int = df2.select_dtypes(include=['int64','float64']).copy()
for col in list(df_int.columns):
    print(df.value_counts(df_int[col]).to_string)

<bound method Series.to_string of ID
1.0       1
970.0     1
982.0     1
981.0     1
980.0     1
         ..
485.0     1
484.0     1
483.0     1
482.0     1
1460.0    1
Length: 1371, dtype: int64>
<bound method Series.to_string of Type_propri
20.0     517
60.0     296
50.0     131
120.0     86
160.0     61
80.0      58
70.0      57
30.0      51
90.0      40
190.0     21
85.0      20
75.0      14
45.0       9
180.0      6
40.0       4
dtype: int64>
<bound method Series.to_string of Taille_propri_m2
7200.0      25
9600.0      23
6000.0      16
8400.0      14
9000.0      12
            ..
9037.0       1
9056.0       1
9066.0       1
9069.0       1
215245.0     1
Length: 1014, dtype: int64>
<bound method Series.to_string of Eval_mater_finition
5.0     365
6.0     360
7.0     314
8.0     166
4.0      90
9.0      43
10.0     17
3.0      14
2.0       2
dtype: int64>
<bound method Series.to_string of Eval_etat_gene
5.0    788
6.0    236
7.0    187
8.0     68
4.0     51
9.0     20
3.0     18
2.

### Convertir les peids carré en mètre carré

In [132]:
#Liste de toute les variables à convertir en mètre carré
lst_m2 = ('Taille_propri_m2','Type_placage_m2','Type_placage_m2','m2_sous-sol','Non_fini_m2_SS',
'Surf_m2_etage1','Surf_m2_etage2','basse_quali_m2','Surf_garage_m2','Surf_terrasse_m2',
'Surf_porche_ouvert_m2','Surf_porche_ferme_m2','Surf_porche_3S_m2','Surf_veranda_m2','Surf_piscine_m2','Surf_habit_m2')

In [133]:
for col in range(len(lst_m2)):
    df2[lst_m2[col]] = round(df2[lst_m2[col]] * 0.092903,0)

C:\Users\100518\AppData\Local\Temp\ipykernel_2052\1379972433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[lst_m2[col]] = round(df2[lst_m2[col]] * 0.092903,0)


### Les valeurs aberrantes 

In [134]:
df2

,ID,Type_propri,Classe_zone_propri,Taille_propri_m2,Quartier,Eval_mater_finition,Eval_etat_gene,Annee_construct,Annee_renov,Type_placage_m2,Eval_mater_ext,Non_fini_m2_SS,m2_sous-sol,Surf_m2_etage1,Surf_m2_etage2,basse_quali_m2,Surf_habit_m2,Nb_salledebain_SS,Nb_salledebain,Nb_demi_salledebain,Nb_chambre,Nb_cuisine,Nb_piece,Nb_foyer,Annee_construct_garage,Nb_voiture_garage,Surf_garage_m2,Surf_terrasse_m2,Surf_porche_ouvert_m2,Surf_porche_ferme_m2,Surf_porche_3S_m2,Surf_veranda_m2,Surf_piscine_m2,Valeur_caract_ext,Mois_vente,Annee_vente,Prix_vente
0,1,60,RL,785.0,CollgCr,7,5,2003,2003,2.0,Gd,14.0,80.0,80.0,79.0,0.0,159.0,1,2,1,3,1,8,0,2003.0,2,51.0,0.0,6.0,0.0,0.0,0.0,0.0,0,2,2008,208500
1,2,20,RL,892.0,Veenker,6,8,1976,1976,0.0,TA,26.0,117.0,117.0,0.0,0.0,117.0,0,2,0,3,1,6,1,1976.0,2,43.0,28.0,0.0,0.0,0.0,0.0,0.0,0,5,2007,181500
2,3,60,RL,1045.0,CollgCr,7,5,2001,2002,1.0,Gd,40.0,85.0,85.0,80.0,0.0,166.0,1,2,1,3,1,6,1,2001.0,2,56.0,0.0,4.0,0.0,0.0,0.0,0.0,0,9,2008,223500
3,4,70,RL,887.0,Crawfor,7,5,1915,1970,0.0,TA,50.0,70.0,89.0,70.0,0.0,160.0,1,1,0,3,1,7,1,1998.0,3,60.0,0.0,3.0,25.0,0.0,0.0,0.0,0,2,2006,140000
4,5,60,RL,1325.0,NoRidge,8,5,2000,2000,3.0,Gd,46.0,106.0,106.0,98.0,0.0,204.0,1,2,1,4,1,9,1,2000.0,3,78.0,18.0,8.0,0.0,0.0,0.0,0.0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,736.0,Gilbert,6,5,1999,2000,0.0,TA,89.0,89.0,89.0,64.0,0.0,153.0,0,2,1,3,1,7,1,1999.0,2,43.0,0.0,4.0,0.0,0.0,0.0,0.0,0,8,2007,175000
1456,1457,20,RL,1224.0,NWAmes,6,6,1978,1988,1.0,TA,55.0,143.0,193.0,0.0,0.0,193.0,1,2,0,3,1,7,2,1978.0,2,46.0,32.0,0.0,0.0,0.0,0.0,0.0,0,2,2010,210000
1457,1458,70,RL,840.0,Crawfor,7,9,1941,2006,0.0,Ex,81.0,107.0,110.0,107.0,0.0,217.0,0,2,0,4,1,9,2,1941.0,1,23.0,0.0,6.0,0.0,0.0,0.0,0.0,2500,5,2010,266500
1458,1459,20,RL,903.0,NAmes,5,6,1950,1996,0.0,TA,0.0,100.0,100.0,0.0,0.0,100.0,1,1,0,2,1,5,0,1950.0,1,22.0,34.0,0.0,10.0,0.0,0.0,0.0,0,4,2010,142125


In [135]:
#On récupère les variables quantitives 
df_int = df2.select_dtypes(include=['int64','float64']).copy()

# Detection des valeurs aberrantes avec la méthode du Z score
for col in list(df_int.columns):
    mean = np.mean(df_int[col])
    std = np.std(df_int[col])

    threshold = 3
    outlier = []
    for i in df2[col]:
        z = (i-mean)/std
        if z > threshold:
            outlier.append(i)
    print('les outliers pour la variable ' + col + ' sont les suivants : ', outlier)
    print('\n')

les outliers pour la variable ID sont les suivants :  []


les outliers pour la variable Type_propri sont les suivants :  [190, 180, 190, 190, 190, 190, 190, 190, 180, 190, 180, 190, 190, 190, 190, 190, 190, 180, 190, 190, 190, 190, 190, 180, 190, 190, 180]


les outliers pour la variable Taille_propri_m2 sont les suivants :  [4670.0, 14772.0, 19997.0, 15297.0, 4934.0, 6574.0, 4945.0, 4328.0, 10698.0, 4971.0, 4236.0, 5935.0, 5314.0]


les outliers pour la variable Eval_mater_finition sont les suivants :  []


les outliers pour la variable Eval_etat_gene sont les suivants :  [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]


les outliers pour la variable Annee_construct sont les suivants :  []


les outliers pour la variable Annee_renov sont les suivants :  []


les outliers pour la variable Type_placage_m2 sont les suivants :  [6.0, 9.0, 10.0, 7.0, 10.0, 6.0, 6.0, 6.0, 8.0, 14.0, 8.0, 6.0, 7.0, 8.0, 7.0, 6.0, 10.0, 6.0, 8.0, 6.0, 7.0, 9.0, 7.0, 7.0, 8.0, 6.0, 12.0, 7.0, 7.0

In [110]:
#Pour la variable type1_m2_SS et m2_sous-sol on vire les valeurs aberrantes 
df2.drop(df2[df2['Non_fini_m2_SS'] > 176].index, inplace=True)
df2.drop(df2[df2['m2_sous-sol'] > 221].index, inplace=True)
df2.drop(df2[df2['Surf_m2_etage1'] > 221].index, inplace=True)
df2.drop(df2[df2['Surf_m2_etage2'] > 168].index, inplace=True)
df2.drop(df2[df2['basse_quali_m2'] > 13].index, inplace=True)
df2.drop(df2[df2['Surf_habit_m2'] > 288].index, inplace=True)
df2.shape

C:\Users\100518\AppData\Local\Temp\ipykernel_2052\2715977164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(df2[df2['Non_fini_m2_SS'] > 176].index, inplace=True)
C:\Users\100518\AppData\Local\Temp\ipykernel_2052\2715977164.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(df2[df2['m2_sous-sol'] > 221].index, inplace=True)
C:\Users\100518\AppData\Local\Temp\ipykernel_2052\2715977164.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

(1331, 38)

In [137]:
# Repartition pour la variable Type_placage_m2
df2["Nb_foyer"] = df2["Nb_foyer"].replace({2: "plus de 1", 3: "plus de 1"})
sns.distplot(df2[df2.nb_foyer > 0].nb_foyer, kde=False)

C:\Users\100518\AppData\Local\Temp\ipykernel_2052\3568744070.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Nb_foyer"] = df2["Nb_foyer"].replace({2: "plus de 1", 3: "plus de 1"})


AttributeError: 'DataFrame' object has no attribute 'nb_foyer'

In [ ]:
#On vire la modalité Fa de la variable Eval_mater_ext
df2 = df2.drop(df2[df2['Eval_mater_ext'] == 'Fa'].index)

#Regroupement de modalité pour la vairbale Type_placage_m2
df2['Type_placage_m2'] = df2['Type_placage_m2'].apply(lambda x: 1 if x > 0 else 0)
df2['Nb_salledebain_SS'] = df2['Nb_salledebain_SS'].apply(lambda x: 1 if x > 0 else 0)

In [91]:
#Regroupement de modalité pour la vairbale Type_placage_m2
df2['Type_placage_m2'] = df2['Type_placage_m2'].apply(lambda x: 1 if x > 0 else 0)

C:\Users\100518\AppData\Local\Temp\ipykernel_2052\3493080276.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Type_placage_m2'] = df2['Type_placage_m2'].apply(lambda x: 1 if x > 0 else 0)


In [66]:
df2.drop(df2[df2['Non_fini_m2_SS'] > 1900].index, inplace=True)
df2.drop(df2[df2['m2_sous-sol'] > 2300].index, inplace=True)
df2.drop(df2[df2['Surf_m2_etage1'] > 2300].index, inplace=True)
df2.drop(df2[df2['Surf_m2_etage2'] > 1800].index, inplace=True)
df2.drop(df2[df2['basse_quali_m2'] > 140].index, inplace=True)
df2.drop(df2[df2['Surf_habit_m2'] > 288].index, inplace=True)
df2.shape

C:\Users\100518\AppData\Local\Temp\ipykernel_2052\4254028588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(df2[df2['Non_fini_m2_SS'] > 1900].index, inplace=True)
C:\Users\100518\AppData\Local\Temp\ipykernel_2052\4254028588.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(df2[df2['m2_sous-sol'] > 2300].index, inplace=True)
C:\Users\100518\AppData\Local\Temp\ipykernel_2052\4254028588.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

(738, 38)

Nous avons ici, pour chacune de nos variables quantitatives les valeurs qui sont aberrantes. Pour traiter ces valeurs, nous avons plusieurs solutions :

    - supprimer les valeurs aberrantes
    - créer des classes pour atténuer l'impact des valeurs aberrantes
    - imputer les valeurs.
    
N'ayant pas beaucoup d'obeservations, nous avons choisis de traiter les valeurs aberrantes

### Regroupement des modalitées

In [81]:
df3 = df2

In [26]:
#On vérifie que les type des variables année sont bien au format numérique 
df3['Annee_construct'] = df3['Annee_construct'].astype(int)

C:\Users\106443\AppData\Local\Temp\ipykernel_9140\3077024769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Annee_construct'] = df3['Annee_construct'].astype(int)


In [37]:
df3['Annee_construct'] = df3['Annee_construct'].astype(int)

for i, row in df3.iterrows():
    if row['Annee_construct'] <= 1900:
        df3.at[i, 'Annee_construct'] = '1872-1900'
    elif row['Annee_construct'] >= 1901 and row['Annee_construct'] <= 1920:
        df3.at[i, 'Annee_construct'] = '1901-1920'
    elif row['Annee_construct'] >= 1921 and row['Annee_construct'] <= 1940:
        df3.at[i, 'Annee_construct'] = '1921-1940'
    elif row['Annee_construct'] >= 1941 and row['Annee_construct'] <= 1960:
        df3.at[i, 'Annee_construct'] = '1941-1960'
    elif row['Annee_construct'] >= 1961 and row['Annee_construct'] <= 1980:
        df3.at[i, 'Annee_construct'] = '1961-1980'
    elif row['Annee_construct'] >= 1981 and row['Annee_construct'] <= 2000:
        df3.at[i, 'Annee_construct'] = '1981-2000'
    else:
        df3.at[i, 'Annee_construct'] = '2001-2010'


C:\Users\106443\AppData\Local\Temp\ipykernel_8356\2472441090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Annee_construct'] = df3['Annee_construct'].astype(int)
C:\Users\106443\AppData\Local\Temp\ipykernel_8356\2472441090.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.at[i, 'Annee_construct'] = '2001-2010'


In [28]:
for i, row in df3.iterrows():
    if row['Annee_renov'] <= 1960:
        df3.at[i, 'Annee_renov'] = '1950-1960'
    elif row['Annee_renov'] >= 1961 and row['Annee_renov'] <= 1970:
        df3.at[i, 'Annee_renov'] = '1961-1970'
    elif row['Annee_renov'] >= 1971 and row['Annee_renov'] <= 1980:
        df3.at[i, 'Annee_renov'] = '1971-1980'
    elif row['Annee_renov'] >= 1981 and row['Annee_renov'] <= 1990:
        df3.at[i, 'Annee_renov'] = '1981-1990'
    elif row['Annee_renov'] >= 1991 and row['Annee_renov'] <= 2000:
        df3.at[i, 'Annee_renov'] = '1991-2000'
    else:
        df3.at[i, 'Annee_renov'] = '2001-2010'

C:\Users\106443\AppData\Local\Temp\ipykernel_9140\3617585560.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.at[i, 'Annee_renov'] = '2001-2010'


In [32]:
for i, row in df3.iterrows():
    if row['Annee_construct_garage'] <= 1920:
        df3.at[i, 'Annee_construct_garage'] = '1900-1920'
    elif row['Annee_construct_garage'] >= 1921 and row['Annee_construct_garage'] <= 1940:
        df3.at[i, 'Annee_construct_garage'] = '1921-1940'
    elif row['Annee_construct_garage'] >= 1941 and row['Annee_construct_garage'] <= 1960:
        df3.at[i, 'Annee_construct_garage'] = '1941-1960'
    elif row['Annee_construct_garage'] >= 1961 and row['Annee_construct_garage'] <= 1980:
        df3.at[i, 'Annee_construct_garage'] = '1961-1980'
    elif row['Annee_construct_garage'] >= 1981 and row['Annee_construct_garage'] <= 2000:
        df3.at[i, 'Annee_construct_garage'] = '1981-2000'
    else:
        df3.at[i, 'Annee_construct_garage'] = '2001-2010'

TypeError: '<=' not supported between instances of 'str' and 'int'

In [86]:
df3.describe()

,ID,Type_propri,Taille_propri_m2,Eval_mater_finition,Eval_etat_gene,Annee_construct,Annee_renov,Type_placage_m2,Non_fini_m2_SS,m2_sous-sol,Surf_m2_etage1,Surf_m2_etage2,basse_quali_m2,Surf_habit_m2,Nb_salledebain_SS,Nb_demi_salledebain_SS,Nb_salledebain,Nb_demi_salledebain,Nb_chambre,Nb_cuisine,Nb_piece,Nb_foyer,Annee_construct_garage,Nb_voiture_garage,Surf_garage_m2,Surf_terrasse_m2,Surf_porche_ouvert_m2,Surf_porche_ferme_m2,Surf_porche_3S_m2,Surf_veranda_m2,Surf_piscine_m2,Valeur_caract_ext,Mois_vente,Annee_vente,Prix_vente
count,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000,1371.000000
mean,731.169949,56.072210,992.832969,6.180890,5.582057,1972.808169,1985.342815,0.948942,570.606127,1072.869438,1175.778264,353.411379,4.159008,1533.348651,0.428884,0.059081,1.577681,0.394602,2.865791,1.037929,6.552152,0.641138,1978.374179,1.869438,500.389497,97.657914,46.994165,21.016047,3.630926,16.038658,2.938001,43.139314,6.330416,2007.811816,185181.890591
std,421.406120,41.063631,949.919852,1.344125,1.081199,29.374036,20.453845,1.623375,443.917548,435.797686,385.409660,439.405368,40.246298,519.313228,0.513906,0.238937,0.547374,0.502200,0.782708,0.194875,1.590350,0.644709,24.688912,0.631086,185.939979,126.759726,64.983737,60.500067,30.241332,57.403495,41.455453,503.065627,2.692423,1.331112,78872.352654
min,1.000000,20.000000,121.000000,2.000000,2.000000,1880.000000,1950.000000,0.000000,0.000000,0.000000,438.000000,0.000000,0.000000,438.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,1900.000000,1.000000,160.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,35311.000000
25%,365.500000,20.000000,718.500000,5.000000,5.000000,1955.000000,1967.500000,0.000000,226.000000,808.000000,894.000000,0.000000,0.000000,1152.000000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,378.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,133950.000000
50%,733.000000,50.000000,891.000000,6.000000,5.000000,1975.000000,1994.000000,0.000000,476.000000,1008.000000,1098.000000,0.000000,0.000000,1479.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,484.000000,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,167000.000000
75%,1094.500000,70.000000,1088.000000,7.000000,6.000000,2001.000000,2004.000000,1.000000,811.000000,1314.000000,1414.000000,736.000000,0.000000,1788.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,582.500000,171.000000,69.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,217750.000000
max,1460.000000,190.000000,19997.000000,10.000000,9.000000,2010.000000,2010.000000,14.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,2.000000,2.000000,3.000000,2.000000,6.000000,3.000000,12.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


## 4. Modélisation 

### Échantillonnage 

In [30]:
#On ne va pas travailler sur la totalité de nos observations, il est important de réaliser un échantillonnage 
#Train set, Test set
y = df['SalePrice'] #On défini la variable a expliquer
y=y.astype('int')
X = df.drop(columns = ["SalePrice"], inplace=True) # On défini nos features

### Dummies

In [31]:
X = pd.get_dummies(X)
X

C:\Users\106443\AppData\Local\Temp\ipykernel_9140\415363062.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  X = pd.get_dummies(X)


TypeError: object of type 'NoneType' has no len()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.80, test_size=0.20, random_state=0, stratify=y) 
# On réalise l'échantillonage en splitant le datasets a 80% pour le train set et 20% pour le test set
# On initialise un random_state égal à 0 pour obtenir toujours le même tirage aléatoire
# On initialise le paramètre de stratification afin de conserver la répartition par strate